In [2]:
import redis
import time


class LimitExceededException(Exception):
    pass

max_rps = 15  # requests per second

# Basic database settings. Don't change this line!
r = redis.StrictRedis(host='localhost', port=6381, db=4) 

In [3]:
def save_sync(r: redis.StrictRedis, uid: str, partner_id: int, partner_uid: str):
    """
        Set the values for the pairs <partner_id, uid> and <partner_id, partner_uid>
        Don't forget to set the ttls which you defined in the function set_ttls

    """
    ttl = r.hget('ttls', partner_id)
    if ttl is None:
        ttl = 0
    pipe = r.pipeline()
    pipe.set('uid-partner:{}:{}'.format(partner_id, uid), partner_uid)
    pipe.expire('uid-partner:{}:{}'.format(partner_id, uid), ttl)
    pipe.set('partner-uid:{}:{}'.format(partner_id, partner_uid), uid)
    pipe.expire('partner-uid:{}:{}'.format(partner_id, partner_uid), ttl)
    
    pipe.execute()

        
def get_partner_uid(r: redis.StrictRedis, uid: str, partner_id: int):
    """Get the partner id by the pair (uid, partner id)
    
    Args:
        r (redis.StrictRedis): redis instance
        uid (str): our uid
        partner_id (str): id of the partner
        
    Examples:
        >>> get_partner_uid(r, 'e5a370cc-6bdc-43ae-baaa-8fd4531847f7', 12)
    """
    restrict_function(r, 'get_partner_uid')
    return r.get('uid-partner:{}:{}'.format(partner_id, uid))


def get_uid(r: redis.StrictRedis, partner_id: int, partner_uid: str):
    """Get the uid by the pair (partner id, partner uid)
    
    Args:
        r (redis.StrictRedis): redis instance
        partner_id (int): id of partner
        partner_uid (str): uid of partner
        
    Examples:
        >>> get_uid(r, 12, '25b6e9a6-fca8-427c-94df-2577e62b2bf0')

    """
    restrict_function(r, 'get_uid')
    return r.get('partner-uid:{}:{}'.format(partner_id, partner_uid))


def set_ttls(r: redis.StrictRedis, ttls):
    """Set the ttl by partner id
       
    Args:
        r (redis.StrictRedis): redis instance
        ttls (dict): dictionary of pairs <partner_id, ttl>
        
    Examples:
        >>> set_ttls(r, {12: 5, 3: 0.25})

    """
    for partner_id, ttl in ttls.items():
        r.hset('ttls', partner_id, ttl)

def restrict_function(r, func_name):
    pipe = r.pipeline()
    if not r.get(func_name):
        pipe.set(func_name, 0)   
        pipe.expire(func_name, 1)
    
    pipe.incrby(func_name, 1)
    pipe.execute()
    c = r.get(func_name)
    if(c and int(c)>max_rps):
        raise LimitExceededException

In [4]:

import time
from random import sample

from uuid import uuid4


def make_sample_data(uid_count, partner_id_count):

    uids = {}
    for i in range(uid_count):
        uid = str(uuid4())
        partners = range(partner_id_count)
        uids[uid] = {p: str(uuid4()) for p in partners}

    return uids


def test1(id: str):

    try:
        partner_id_count = 5

        r.flushdb()
        sample_data = make_sample_data(
            uid_count=5,
            partner_id_count=5
        )

        ttls = {p_id: 10 for p_id in range(partner_id_count)}

        set_ttls(r, ttls)
        for uid, partners in sample_data.items():
            for partner_id, partner_uid in partners.items():
                save_sync(r, uid, partner_id, partner_uid)
        for uid, partners in sample_data.items():
            for partner_id, partner_uid in partners.items():
                assert partner_uid == get_partner_uid(r, uid, partner_id).decode('utf-8'), 'partner_uid missing'
                assert uid == get_uid(r, partner_id, partner_uid).decode('utf-8'), 'uid missing'
        print(f'Test 1: ok, id={id}')
    except Exception:
        print('Test 1: incorrect answer')



In [5]:

def test2(id: str):

    try:
        r.flushdb()
        sample_data = make_sample_data(
            uid_count=3,
            partner_id_count=3
        )

        ttls = {0: 0, 1: 1, 2: 3}

        set_ttls(r, ttls)

        for uid, partners in sample_data.items():
            for partner_id, partner_uid in partners.items():
                save_sync(r, uid, partner_id, partner_uid)

        for uid, partners in sample_data.items():
            for partner_id in [0]:
                partner_uid = partners[partner_id]
                assert get_partner_uid(r, uid, partner_id) is None
                assert get_uid(r, partner_id, partner_uid) is None

            for partner_id in [1, 2]:
                partner_uid = partners[partner_id]
                assert partner_uid == get_partner_uid(r, uid, partner_id).decode('utf-8')
                assert uid == get_uid(r, partner_id, partner_uid).decode('utf-8')

        time.sleep(1)
        for uid, partners in sample_data.items():
            for partner_id in [0, 1]:
                partner_uid = partners[partner_id]
                assert get_partner_uid(r, uid, partner_id) is None
                assert get_uid(r, partner_id, partner_uid) is None

            for partner_id in [2]:
                partner_uid = partners[partner_id]
                assert partner_uid == get_partner_uid(r, uid, partner_id).decode('utf-8')
                assert uid == get_uid(r, partner_id, partner_uid).decode('utf-8')

        for uid, partners in sample_data.items():
            for partner_id in [1, 2]:
                partner_uid = partners[partner_id]
                save_sync(r, uid, partner_id, partner_uid)

        for uid, partners in sample_data.items():
            for partner_id in [0]:
                partner_uid = partners[partner_id]
                assert get_partner_uid(r, uid, partner_id) is None
                assert get_uid(r, partner_id, partner_uid) is None

            for partner_id in [1, 2]:
                partner_uid = partners[partner_id]
                assert partner_uid == get_partner_uid(r, uid, partner_id).decode('utf-8')
                assert uid == get_uid(r, partner_id, partner_uid).decode('utf-8')

        time.sleep(2)

        for uid, partners in sample_data.items():
            for partner_id in [0, 1]:
                partner_uid = partners[partner_id]
                assert get_partner_uid(r, uid, partner_id) is None
                assert get_uid(r, partner_id, partner_uid) is None

            for partner_id in [2]:
                partner_uid = partners[partner_id]
                assert partner_uid == get_partner_uid(r, uid, partner_id).decode('utf-8')
                assert uid == get_uid(r, partner_id, partner_uid).decode('utf-8')
        print(f'Test 2: ok, id={id}')
    except Exception:
        print('Test 2: incorrect answer')



In [6]:

def test3(id: str):

    r.flushdb()
    try:
        sample_data = make_sample_data(
            uid_count=2,
            partner_id_count=2
        )

        first_uid, second_uid = list(sample_data)

        ttls = {0: 1, 1: 1}
        set_ttls(r, ttls)

        for partner_id, partner_uid in sample_data[first_uid].items():
            save_sync(r, first_uid, partner_id, partner_uid)

        ttls_upd = {0: 3}
        set_ttls(r, ttls_upd)
        for partner_id, partner_uid in sample_data[second_uid].items():
            save_sync(r, second_uid, partner_id, partner_uid)

        for uid, partners in sample_data.items():
            for partner_id in [0, 1]:
                partner_uid = partners[partner_id]
                assert partner_uid == get_partner_uid(r, uid, partner_id).decode('utf-8')
                assert uid == get_uid(r, partner_id, partner_uid).decode('utf-8')

        time.sleep(2)

        partners = sample_data[second_uid]
        partner_id = 0
        partner_uid = partners[partner_id]
        assert partner_uid == get_partner_uid(r, second_uid, partner_id).decode('utf-8')
        assert second_uid == get_uid(r, partner_id, partner_uid).decode('utf-8')

        for uid, partners in sample_data.items():
            for partner_id, partner_uid in partners.items():
                if uid == second_uid and partner_id == 0:
                    continue
                assert get_partner_uid(r, uid, partner_id) is None
                assert get_uid(r, partner_id, partner_uid) is None

        print(f'Test 3: ok, id={id}')
    except Exception:
        print('Test 3: incorrect answer')



In [7]:

def test4(id: str):

    r.flushdb()
    try:
        sample_data = make_sample_data(
            uid_count=2,
            partner_id_count=2
        )
        ttls = {1: 2}

        set_ttls(r, ttls)

        for uid, partners in sample_data.items():
            for partner_id, partner_uid in partners.items():
                save_sync(r, uid, partner_id, partner_uid)

        time.sleep(1)

        for uid, partners in sample_data.items():
            for partner_id, partner_uid in partners.items():
                if partner_id == 0:
                    assert get_partner_uid(r, uid, partner_id) is None
                    assert get_uid(r, partner_id, partner_uid) is None
                else:
                    assert partner_uid == get_partner_uid(r, uid, partner_id).decode('utf-8')
                    assert uid == get_uid(r, partner_id, partner_uid).decode('utf-8')

        print(f'Test 4: ok, id={id}')
    except Exception:
        print('Test 4: incorrect answer')



In [8]:

def test5(id: str):
    try:
        start = time.time()
        r.flushdb()

        partner_id_count = 500

        uids = {}
        for i in range(1000):
            uid = str(uuid4())
            partners = sample(range(partner_id_count), 5)
            uids[uid] = {p: str(uuid4()) for p in partners}

        ttls = {i: 5 for i in range(500)}

        for i in range(501, 1000):
            ttls[i] = 100

        set_ttls(r, ttls)

        for uid, partners in uids.items():
            for partner_id, partner_uid in partners.items():
                save_sync(r, uid, partner_id, partner_uid)

        time.sleep(5)
        bad_count = 0
        for uid, partners in uids.items():
            for partner_id, partner_uid in partners.items():
                if get_partner_uid(r, uid, partner_id) is None:
                    bad_count += 1

        assert abs(bad_count - 5000) / 5000 < 0.01
        end = time.time()
        assert end - start < 120
        print(f'Test 5: ok, id={id}')
    except Exception:
        print('Test 5: incorrect answer')



In [9]:
def test6(id: str):

    try:

        r.flushdb()
        times = 1
        for i in range(150):
                    try:
                        get_partner_uid(r, '0cae0c53-dcbb-40e6-9552-0f385a7746fb', 1)
                    except LimitExceededException:
                        times+=1
                        time.sleep(1)
        assert times==10, 'problem in function limitation'

        print(f'Test 6: ok, id={id}')
    except Exception as e:
        print(e)
        print("exception")
        print('Test 6: incorrect answer')

In [10]:
test1('852ccfc9-97e7-4e38-bd9d-cb3c485af374')

Test 1: incorrect answer


In [11]:
test2('852ccfc9-97e7-4e38-bd9d-cb3c485af374')

Test 2: incorrect answer


In [12]:
test3('852ccfc9-97e7-4e38-bd9d-cb3c485af374')

Test 3: ok, id=852ccfc9-97e7-4e38-bd9d-cb3c485af374


In [13]:
test4('852ccfc9-97e7-4e38-bd9d-cb3c485af374')

Test 4: ok, id=852ccfc9-97e7-4e38-bd9d-cb3c485af374


In [14]:
max_rps=100000
test5('852ccfc9-97e7-4e38-bd9d-cb3c485af374')

Test 5: ok, id=852ccfc9-97e7-4e38-bd9d-cb3c485af374


In [16]:
max_rps=15
test6('852ccfc9-97e7-4e38-bd9d-cb3c485af374')

Test 6: ok, id=852ccfc9-97e7-4e38-bd9d-cb3c485af374
